In [1]:
import pandas as pd
import numpy as np

In [3]:
import seaborn as sns
# tips 데이터셋 로드
df = sns.load_dataset('tips')

x = df.drop(['tip'], axis=1) 
y = df['tip']


# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2023)

x_test = pd.DataFrame(x_test.reset_index())
x_train = pd.DataFrame(x_train.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

x_test.rename(columns={'index':'cust_id'}, inplace=True)
x_train.rename(columns={'index':'cust_id'}, inplace=True)
y_train.columns = ['cust_id', 'target']

- 레스토랑의 tip 예측 문제
- 데이터의 결측치,이상치,변수에 대해 처리하고
- 회귀모델을 사용하여 Rsq, MSE 값을 산출하시오.

- 데이터셋 설명
- total_bill(총청구액):손님의 식사 총 청구액
- tip(팁):팁의 양 --> 종속변수
- sex(성별):손님의 성별
- smoker(흡연자):손님의 흡연 여부(Yes, No)
- day(요일):식사가 이루어진 요일
- time(시간):점심 또는 저녁 중 언제 식사가 이루어졌는지
- size(인원수):식사에 참석한 인원 수

- 2. 데이터 탐색(EDA)

In [4]:
# 데이터의 행/열 확인

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(195, 7)
(49, 7)
(195, 2)


In [5]:
# 초기 데이터 확인

print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

   cust_id  total_bill     sex smoker  day    time  size
0      158       13.39  Female     No  Sun  Dinner     2
1      186       20.90  Female    Yes  Sun  Dinner     3
2       21       20.29  Female     No  Sat  Dinner     2
   cust_id  total_bill     sex smoker  day    time  size
0      154       19.77    Male     No  Sun  Dinner     4
1        4       24.59  Female     No  Sun  Dinner     4
2       30        9.55    Male     No  Sat  Dinner     2
   cust_id  target
0      158    2.61
1      186    3.50
2       21    2.75


In [6]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인해보세요

print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cust_id     195 non-null    int64   
 1   total_bill  195 non-null    float64 
 2   sex         195 non-null    category
 3   smoker      195 non-null    category
 4   day         195 non-null    category
 5   time        195 non-null    category
 6   size        195 non-null    int64   
dtypes: category(4), float64(1), int64(2)
memory usage: 6.0 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cust_id     49 non-null     int64   
 1   total_bill  49 non-null     float64 
 2   sex         49 non-null     category
 3   smoker      49 non-null     category
 4   day         49 non-null     category
 5   time        49 non-null     category
 6   size   

In [7]:
# x_train과 x_test 데이터의 기초통계량을 잘 비교해보세요

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

          cust_id  total_bill        size
count  195.000000  195.000000  195.000000
mean   122.056410   20.054667    2.543590
std     70.668034    8.961645    0.942631
min      0.000000    3.070000    1.000000
25%     59.500000   13.420000    2.000000
50%    121.000000   17.920000    2.000000
75%    182.500000   24.395000    3.000000
max    243.000000   50.810000    6.000000
          cust_id  total_bill       size
count   49.000000   49.000000  49.000000
mean   119.285714   18.716531   2.673469
std     70.918674    8.669864   0.987162
min      2.000000    5.750000   2.000000
25%     62.000000   12.740000   2.000000
50%    123.000000   16.660000   2.000000
75%    180.000000   21.010000   3.000000
max    239.000000   44.300000   6.000000
          cust_id      target
count  195.000000  195.000000
mean   122.056410    3.021692
std     70.668034    1.402690
min      0.000000    1.000000
25%     59.500000    2.000000
50%    121.000000    2.920000
75%    182.500000    3.530000
max    243.00

In [9]:
# object, category 데이터도 추가 확인
# print(x_train.describe(include='object'))
# print(x_test.describe(include='object'))

print(x_train.describe(include='category'))
print(x_test.describe(include='category'))

         sex smoker  day    time
count    195    195  195     195
unique     2      2    4       2
top     Male     No  Sat  Dinner
freq     125    120   71     142
         sex smoker  day    time
count     49     49   49      49
unique     2      2    4       2
top     Male     No  Sat  Dinner
freq      32     31   16      34


In [10]:
# y데이터도 구체적으로 살펴보세요
print(y_train.head())

   cust_id  target
0      158    2.61
1      186    3.50
2       21    2.75
3       74    2.20
4       43    1.32


In [11]:
# y데이터도 구체적으로 살펴보세요
print(y_train.describe().T)

         count        mean        std  min   25%     50%     75%    max
cust_id  195.0  122.056410  70.668034  0.0  59.5  121.00  182.50  243.0
target   195.0    3.021692   1.402690  1.0   2.0    2.92    3.53   10.0


- 3. 데이터 전처리 및 분리
- 1) 결측치 2) 이상치 3) 변수 처리하기

In [12]:
# 결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id       0
total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
cust_id       0
total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
cust_id    0
target     0
dtype: int64


In [13]:
# 변수처리

# 불필요한 변수(columns) 제거
# cust_id는 불필요한 변수이므로 제거합니다.
# 단, test셋의 cust_id가 나중에 제출이 필요하다면 별도로 저장해둠

cust_id = x_test['cust_id'].copy()

# 각 데이터에서 cust_id 변수 제거
x_train = x_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])

In [14]:
# 변수처리(원핫인코딩)
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  195 non-null    float64 
 1   sex         195 non-null    category
 2   smoker      195 non-null    category
 3   day         195 non-null    category
 4   time        195 non-null    category
 5   size        195 non-null    int64   
dtypes: category(4), float64(1), int64(1)
memory usage: 4.5 KB
None


In [15]:
# 변수처리(원핫인코딩)
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
print(x_train.info())
print(x_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total_bill   195 non-null    float64
 1   size         195 non-null    int64  
 2   sex_Male     195 non-null    uint8  
 3   sex_Female   195 non-null    uint8  
 4   smoker_Yes   195 non-null    uint8  
 5   smoker_No    195 non-null    uint8  
 6   day_Thur     195 non-null    uint8  
 7   day_Fri      195 non-null    uint8  
 8   day_Sat      195 non-null    uint8  
 9   day_Sun      195 non-null    uint8  
 10  time_Lunch   195 non-null    uint8  
 11  time_Dinner  195 non-null    uint8  
dtypes: float64(1), int64(1), uint8(10)
memory usage: 5.1 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total_bill   49 non-null     float64
 1   size   

In [16]:
# 데이터 분리

# 데이터를 훈련 세트와 검증용 세트로 분할 (80% 훈련, 20% 검증용)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['target'],
                                                 test_size=0.2,
                                                 random_state=23)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(156, 12)
(39, 12)
(156,)
(39,)


- 4. 모델링 및 성능평가

In [17]:
# 랜덤포레스트 모델 사용 (참고 : 분류모델은 RandomForestClassifier)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=2023)
model.fit(x_train,y_train)

RandomForestRegressor(random_state=2023)

In [18]:
# 모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [19]:
# 모델 성능 평가 (R-squared, MSE 등)
from sklearn.metrics import r2_score, mean_squared_error
r2 = r2_score(y_val, y_pred)            # 실제값, 예측값
mse = mean_squared_error(y_val, y_pred) # 실제값, 예측값

In [20]:
# MSE
print(mse)

0.9812277338461534


In [21]:
# RMSE
rmse = mse ** 0.5
print(rmse)

0.990569398803614


In [22]:
# RSQ
print(r2)

0.4286497615634072


- 5. 예측값 제출
- (주의) x_test를 모델에 넣어 나온 예측값을 제출해야 함

In [23]:
# (실기시험 안내사항)
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id':cust_id,'target':y_result}).to_csv('003000000.csv', index=False)

# 모델을 사용하여 테스트 데이터 예측
y_result = model.predict(x_test)
result = pd.DataFrame({'cust_id':cust_id,'target':y_result})
print(result[:5])

   cust_id  target
0      154  3.2266
1        4  4.1160
2       30  1.8966
3       75  1.8735
4       33  3.0267


In [24]:
# tip : 데이터를 저장한다음 불러와서 제대로 제출했는지 확인해보자
# pd.DataFrame({'cust_id':cust_id,'target':y_result}).to_csv('003000000.csv', index=False)
# df2 = pd.read_csv("수험번호.csv")
# print(df2.head())